In [11]:
# execute as needed
# !pip install -r requirements.txt

## Process Books

In [6]:
import sys
import warnings

sys.path.append("../")

RAW_BOOKS_FOLDER_PATH = "../raw_data/books/"
PROCESSED_BOOKS_FOLDER_PATH = "../processed_data/books/"

warnings.filterwarnings('ignore')

In [2]:
import os
import fitz
from modules.embedding_tracking import add_new_file, file_exists

available_files = os.listdir(RAW_BOOKS_FOLDER_PATH)

for i in available_files:
    print(f"Processing {i}")
    
    processed_fname = ".".join(i.split(".")[:-1])+".txt"
    print(processed_fname)
    
    if file_exists(processed_fname):
        print(f"Skipping {i} as it has already been processed.")
        continue

    text = ""
    if i.endswith(".pdf"):
        fname = os.path.join(RAW_BOOKS_FOLDER_PATH, i)
        pdf_document = fitz.open(fname)
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            page_text = page.get_text()
            # Fix spaces between words
            page_text = " ".join(page_text.replace("\n", " ").split())
            text += page_text + "\n"
        pdf_document.close()
            
    else:
        # invalid file types
        print(f"Skipping {i} as it is not a valid file.")
        continue
    
    with open(os.path.join(PROCESSED_BOOKS_FOLDER_PATH, processed_fname), "w", encoding="utf-8") as f:
        f.write(text)
        add_new_file(processed_fname)
        
    print(f"Finished processing {i}.")

Processing .DS_Store
.txt
Skipping .DS_Store as it is not a valid file.
Processing Sunrise on the Reaping.pdf
Sunrise on the Reaping.txt
Skipping Sunrise on the Reaping.pdf as it has already been processed.
Processing All Fours (Miranda July).pdf
All Fours (Miranda July).txt
Skipping All Fours (Miranda July).pdf as it has already been processed.


### Store to storage as vectors

In [7]:
# https://medium.com/the-ai-forum/semantic-chunking-for-rag-f4733025d5f5

from modules import accessor, embedding_tracking
from langchain.text_splitter import RecursiveCharacterTextSplitter
from nltk.tokenize import sent_tokenize
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np

nltk.download("punkt")

filenames = os.listdir(PROCESSED_BOOKS_FOLDER_PATH)
embedding_functions = list(accessor.EMBEDDING_FUNCTIONS.keys())

for filename in filenames:
    with open(os.path.join(PROCESSED_BOOKS_FOLDER_PATH, filename), "r", encoding="utf-8") as file:
        text = file.read()
        print(f"Tokenizing {filename}")
        
        for embedding_function in embedding_functions:
            # # Full text embedding
            # suffix = "full_text"
            # if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
            #     try:
            #         ids = accessor.insert(
            #             data=[text],
            #             embedding_func=embedding_function,
            #             custom_suffix=suffix,
            #         )
            #         embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
            #     except Exception as e:
            #         print(f"Error storing {suffix} for {filename}: {e}")
            # else:
            #     print(f"Skipping {filename} for {embedding_function} as it has already been processed with suffix {suffix}.")
                    
            # Fixed size chunk text embedding
            suffix = "512_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                chunked_texts = [text[i:i+512] for i in range(0, len(text), 512)]
                try:
                    ids = accessor.insert(
                        data=chunked_texts,
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
            else:
                print(f"Skipping {filename} for {embedding_function} as it has already been processed with suffix {suffix}.")
                
            # Recursive character text splitting
            suffix = "recursive_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=512,
                    chunk_overlap=100,
                    length_function=len,
                )
                chunks = text_splitter.split_text(text)
                try:
                    ids = accessor.insert(
                        data=chunks,
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
                    
            # Separator text splitting
            suffix = "separator_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=512,
                    chunk_overlap=100,
                    length_function=len,
                    separators=["CHAPTER", "\n\n", "\n", " ", ""],
                )
                chunks = text_splitter.split_text(text)
                try:
                    ids = accessor.insert(
                        data=chunks,
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
            
            # Semantic chunking
            suffix = "semantic_chunks"
            if not embedding_tracking.is_file_processed(filename, "{}_{}".format(embedding_function, suffix)):
                sentences = sent_tokenize(text)
                threshold = 0.7 # can be adjusted
                chunks = []
                curr_chunk = [sentences[0]]
                for i in range(0, len(sentences)-1):
                    embeddings_1 = accessor.EMBEDDING_FUNCTIONS.get(embedding_function).embed_query(sentences[i])
                    embeddings_2 = accessor.EMBEDDING_FUNCTIONS.get(embedding_function).embed_query(sentences[i+1])
                    
                    emb_1 = np.array(embeddings_1)
                    emb_2 = np.array(embeddings_2)

                    if np.linalg.norm(emb_1) == 0 or np.linalg.norm(emb_2) == 0:
                        continue

                    try:
                        similarity = cosine_similarity([emb_1], [emb_2])[0][0]
                    except Exception as e:
                        print(f"Error calculating similarity for {sentences[i]} and {sentences[i+1]}: {e}")
                        continue

                    if similarity > threshold:
                        chunks.append(curr_chunk)
                        curr_chunk = [sentences[i+1]]
                        continue
                    
                    curr_chunk.append(sentences[i+1])
                
                try:
                    ids = accessor.insert(
                        data=[" ".join(chunk) for chunk in chunks],
                        embedding_func=embedding_function,
                        custom_suffix=suffix,
                    )
                    embedding_tracking.mark_file_processed(filename, "{}_{}".format(embedding_function, suffix))
                except Exception as e:
                    print(f"Error storing {suffix} for {filename}: {e}")
            
    print(f"Finished processing {filename} for embeddings.")
    
# LLM based chunking
suffix = "llm_chunked"
llm_chunked_fnames = ["all_fours_txt_chunks.json", "sunrise_on_the_reaping_chunks.json"]
for fname in llm_chunked_fnames:
    embedding_tracking.add_new_file(fname)
    
for embedding_function in embedding_functions:
    for fname in llm_chunked_fnames:
        if not embedding_tracking.is_file_processed(fname, "{}_{}".format(embedding_function, suffix)):
            try:
                with open(os.path.join("../llm_chunked_data", fname), "r", encoding="utf-8") as f:
                    chunks = json.load(f)
                
                ids = accessor.insert(
                    data=chunks,
                    embedding_func=embedding_function,
                    custom_suffix=fname,
                )
                embedding_tracking.mark_file_processed(fname, "{}_{}".format(embedding_function, suffix))
            except Exception as e:
                print(f"Error storing {suffix} for {fname}: {e}")
        else:
            print(f"Skipping {fname} for {embedding_function} as it has already been processed with suffix {suffix}.")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yudhistiraonggowarsito/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenizing All Fours (Miranda July).txt
Skipping All Fours (Miranda July).txt for nomic-embed-text as it has already been processed with suffix 512_chunks.
Skipping All Fours (Miranda July).txt for mxbai-embed-large as it has already been processed with suffix 512_chunks.
Inserted data with IDs: ['9ffad6a2-9b60-477c-9479-b42892b76bfb', '87735f7b-cdea-4daa-9319-cc6e8143c983', '87cc5b2b-4ae6-400b-9258-96da1244211c', '0c0eac39-a294-4ede-b13c-00b8302b725e', 'fa034caa-16b9-45f9-adaa-8cdc5af5b262', '2fe4918b-e34f-4a50-a22b-3377441fcdd3', '977a4412-75f2-4b0c-8c02-afb31f9811cc', 'f3b0f879-c989-43c4-915b-c11355241ffe', '2ab16b40-c750-4d93-b500-57c6b41ef9f3', '5144047d-cb13-4fdf-95de-e3409e6a5886', '498bcafa-668d-493a-bb91-463b0ed438ae', '04357f2e-d847-4cb6-9be6-d7578dbbef0b', '05a76dc4-173e-4b54-ad6e-61dc04e1e8c3', '1b43f6af-64d5-46b6-94a3-4d880983ea21', '5e42bbf3-e5e4-4ef6-a46b-64481d33df2b', 'a1dc98af-49fa-4181-a0f3-7d39d42c54cc', 'af0db61b-71b4-4eca-9a25-6d5d6017e719', '0fd15bc3-a710-45dc-89

In [12]:
available_collections = accessor.list_collections()
print(f"Available collections: {available_collections}")

documents = accessor.get(
    query="What book has the word Sunrise in it?",
    embedding_func="nomic-embed-text",
    custom_suffix="recursive_chunks",
    n_results=10
)
for i in documents:
    print(i.page_content)
    print("-------------------------")

Available collections: ['mxbai-embed-large_collection_separator_chunks', 'mxbai-embed-large_collection_all_fours_txt_chunks.json', 'bge-m3_collection_recursive_chunks', 'mxbai-embed-large_collection_semantic_chunks', 'nomic-embed-text_collection_recursive_chunks', 'langchain', 'bge-m3_collection_512_chunks', 'nomic-embed-text_collection_llm_chunked', 'bge-m3_collection_sunrise_on_the_reaping_chunks.json', 'mxbai-embed-large_collection_recursive_chunks', 'bge-m3_collection_all_fours_txt_chunks.json', 'nomic-embed-text_collection_all_fours_txt_chunks.json', 'nomic-embed-text_collection_512_chunks', 'nomic-embed-text_collection_semantic_chunks', 'nomic-embed-text_collection_sunrise_on_the_reaping_chunks.json', 'mxbai-embed-large_collection_512_chunks', 'nomic-embed-text_collection_separator_chunks', 'bge-m3_collection_separator_chunks', 'mxbai-embed-large_collection_sunrise_on_the_reaping_chunks.json', 'bge-m3_collection_semantic_chunks']
Sunrise on the Reaping, and I hope my English-majo